In [ ]:
!pip install scikit-surprise


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 7.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3095443 sha256=bb4dd412aca206aa93d009ba2791918632315bd218276103ae8266ffe569387d
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [ ]:
import pandas as pd
from surprise import Dataset, Reader

# Load the Excel file into a pandas DataFrame
df = pd.read_excel('miniData.xlsx', engine='openpyxl')

# Convert the DataFrame to a CSV file with UTF-8 encoding
df.to_csv('DataPi.csv', index=False, encoding='utf-8')



# Load the dataset from the CSV file using Pandas
df = pd.read_csv('DataPi.csv')

df['level'] = df['level'].apply(lambda x: float(x.rstrip('th')))




# Define the format for the dataset
reader = Reader(line_format='user item rating', sep=',', skip_lines=1, rating_scale=(1, 10))

# Remove leading spaces from the column names
df.columns = df.columns.str.strip()

# Load the dataset from the Pandas dataframe
data = Dataset.load_from_df(df[['Question', 'Answer', 'level']], reader)



# display the DataFrame
print(df['Answer'])



0        3
1        3
2        2
3        4
4        4
      ... 
254     76
255     92
256     42
257    161
258    164
Name: Answer, Length: 259, dtype: int64


In [ ]:
# count the number of rows for each value of the 'code' column
counts = df['level'].value_counts()

# print the counts
print(counts)

2.0    232
1.0     27
Name: level, dtype: int64


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving miniData.xlsx to miniData.xlsx


In [ ]:
matching_rows = df.loc[df['Question'] == 'What is the difference between 99 and 56?']

# Print the matching row(s)
print(matching_rows)

Empty DataFrame
Columns: [Question, Answer, level]
Index: []


In [ ]:
from surprise import KNNBasic
from surprise import Dataset
from surprise import Reader
import pandas as pd



# Get a list of all questions with the desired level
level = 4.0
questions = df[df['level'] == level]['Question'].tolist()

# Create a dataset with 3 columns: question, answer, level
new_ratings = [(0, q, 0.001) for q in questions]
reader = Reader(line_format='user item rating', sep=',')
new_data = Dataset.load_from_df(pd.DataFrame(new_ratings, columns=['Question', 'Answer', 'level']), reader)

# Define the algorithm
sim_options = {'name': 'cosine', 'user_based': False}
algo = KNNBasic(sim_options=sim_options)

# Train the algorithm on the new data
trainset = new_data.build_full_trainset()
algo.fit(trainset)

# Define a function to get the next question for a user based on their level
def get_next_question(data, user_level):
    # Filter the data by the user's level
    filtered_data = data[data['level'] == user_level]
    # Randomly choose a question from the filtered data
    row = filtered_data.sample()
    return row['Question'].iloc[0], row['Answer'].iloc[0]

# Define a function to update the level of a user based on their performance
def update_user_level(user_level, correct, attempts):
    # If the user got the question correct, keep the same level
    if correct:
        # If user has attempted 5 questions correctly, increase their level
        if attempts % 5 == 0:
            if user_level == 1.0:
                return 2.0
            elif user_level == 2.0:
                return 3.0
            elif user_level == 3.0:
                return 4.0
            else:
                return 4.0
        else:
            return user_level
    else:
        # If user has attempted 5 questions incorrectly, decrease their level
        if attempts % 5 == 0:
            if user_level == 4.0:
                return 3.0
            elif user_level == 3.0:
                return 2.0
            elif user_level == 2.0:
                return 1.0
            else:
                return 1.0
        else:
            return user_level


# Test the functions
user_level = 1.0
attempts = 0
correctt = 0
while attempts < 5:
  question, answer = get_next_question(df, user_level)
  print('Question:', question)
  print('Answer:', answer)

  answer1 = input("Enter your answer: ")
  answer = str(answer)
  print('Answer:', answer)
  if answer1 == answer:
      correct= True
      print('Correct!')
      correctt += 1
      user_level = update_user_level(user_level, correct,attempts)
  else:

      correct= False
      print('Incorrect!')
      user_level = update_user_level(user_level, correct,attempts)
  attempts += 1
  #correct = input('Did you get it correct? (y/n)') == 'y'
  
 
print('New level:', user_level)

# Get recommendations for the user's current level
questions = df[df['level'] == user_level]['Question'].tolist()
new_ratings = [(0, q, 0.001) for q in questions]
new_data = Dataset.load_from_df(pd.DataFrame(new_ratings, columns=['Question', 'Answer', 'level']), reader)
trainset = new_data.build_full_trainset()
algo.fit(trainset)

for q in questions:
    item_id = trainset.to_inner_iid(q)
    neighbors = algo.get_neighbors(item_id, k=10)
    top_10_recommendations = [trainset.to_raw_iid(n) for n in neighbors]
    print(f"Recommendations for '{q}':")
    for r in top_10_recommendations:
        print(f" - {r}")


Computing the cosine similarity matrix...
Done computing similarity matrix.
Question:  If you have 23 candies and you give 15 to your sister, how many candies do you have left?
Answer: 8
Enter your answer: 8
Answer: 8
Correct!
Question: You have 400 kyats. If you  did mowed 9 kyats, how many kyats do you have left?
Answer: 391
Enter your answer: 391
Answer: 391
Correct!
Question: 83 chameleons are in a box. If you did harvested 1 chameleons, how many chameleons are left in the box?
Answer: 82
Enter your answer: 82
Answer: 82
Correct!
Question: There are 64 continental drifts in a bag. If you did found 61 continental drifts, how many continental drifts are left in the bag?
Answer: 3
Enter your answer: 3
Answer: 3
Correct!
Question: There are 51 Handys in a bag. If you did watered 6 Handys, how many Handys are left in the bag?
Answer: 45
Enter your answer: 45
Answer: 45
Correct!
New level: 2.0
Computing the cosine similarity matrix...
Done computing similarity matrix.
Recommendations for

In [ ]:
from surprise import KNNBasic
from surprise import Dataset
from surprise import Reader
import pandas as pd

# Create a dictionary to keep track of incorrect answers
incorrect_answers = {}

def get_next_question(data):
    global incorrect_answers
    # If the user has answered any questions incorrectly before
    if incorrect_answers:
        # Find the most frequent incorrect answer
        most_frequent = set(df[df['Answer'] == False]['Answer'].tolist())
        # Filter the data by the most frequent incorrect answer
        filtered_data = data[data['Answer'] == most_frequent]
        # Check if the filtered data is empty
        if filtered_data.empty:
            # Choose a random question from the entire dataset
            row = data.sample()
            return row['Question'].iloc[0], row['Answer'].iloc[0]
        else:
            # Randomly choose a question from the filtered data
            row = filtered_data.sample()
            return row['Question'].iloc[0], row['Answer'].iloc[0]
    # If the user has not answered any questions incorrectly before, choose a random question
    else:
        row = data.sample()
        return row['Question'].iloc[0], row['Answer'].iloc[0]


# Define a function to update the incorrect answer dictionary
def update_incorrect_answers(answer, correct):
    global incorrect_answers
    if not correct:
        if answer in incorrect_answers:
            incorrect_answers[answer] += 1
        else:
            incorrect_answers[answer] = 1

# Load the data

# Define the algorithm
sim_options = {'name': 'cosine', 'user_based': False}
algo = KNNBasic(sim_options=sim_options)

# Train the algorithm on the data
reader = Reader(line_format='user item rating', sep=',')
data = Dataset.load_from_df(df[['Question', 'Answer', 'level']], reader)
trainset = data.build_full_trainset()
algo.fit(trainset)

# Test the functions
attempts = 0
correctt = 0
while attempts < 5:
    question, answer = get_next_question(df)
    print('Question:', question)
    print('Answer:', answer)
    answer=str(answer)
    answer1 = input("Enter your answer: ")
    print('Answer:', answer)
    if answer1 == answer:
        correct = True
        print('Correct!')
        correctt += 1
    else:
        correct = False
        print('Incorrect!')
        update_incorrect_answers(answer, correct)

    attempts += 1

print('Most frequent incorrect answers:', incorrect_answers)
# Get recommendations for the user's most frequent incorrect answers
if incorrect_answers:
    for incorrect_answer in incorrect_answers:
        matching_rows = df[df['Answer'] != incorrect_answer]
        if not matching_rows.empty:
            # Get a list of questions for the incorrect answer
            questions = matching_rows['Question'].tolist()
            # Create new ratings for the questions and add them to the dataset
            new_ratings = [(0, q, 0.001) for q in questions]
            new_data = Dataset.load_from_df(pd.DataFrame(new_ratings, columns=['Question', 'Answer', 'level']), reader)
            trainset = new_data.build_full_trainset()
            algo.fit(trainset)
            # Get recommendations for the questions
            for q in questions:
                item_id = trainset.to_inner_iid(q)
                neighbors = algo.get_neighbors(item_id, k=4)[1:]  # exclude the first item which is the question itself
                top_10_recommendations = [trainset.to_raw_iid(n) for n in neighbors]
                print(f"Recommendations for '{q}':")
                for r in top_10_recommendations:
                    print(f" - {r}")
        else:
            print(f"No questions found for incorrect answer: {incorrect_answer}")




Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
 - There are 10 bananas in a basket. If you eat 3 bananas, how many bananas are left in the basket?
 -  If you have 11 pencils and you give 4 to your teacher, how many pencils do you have left?
 -  There are 12 cookies on a plate. If you eat 2 cookies, how many cookies are left on the plate?
 - If you have 13 balloons and you give 5 to your friend, how many balloons do you have left?
Recommendations for '225 markkas are in a box. If you did ironed 44 markkas, how many markkas are left in the box?':
 -  There are 5 birds on a tree. If 2 birds fly away, how many birds are left on the tree?
 -  If you have 3 crayons and you give 1 to your friend, how many crayons do you have left?
 -  There are 6 flowers in a vase. If you take 2 flowers out, how many flowers are left in the vase?
 -  If you have 7 stickers and you give 3 to your sister, how many stickers do you have left?
 -  There are 8 books on a shelf. If you

In [ ]:
import re

# Define mistake types using regular expressions
mistake_types = {
    'Addition': r'\d+\s*\+\s*\d+',
    'Subtraction': r'\d+\s*-\s*\d+',
    'Multiplication': r'\d+\s*\*\s*\d+',
    'Division': r'\d+\s*/\s*\d+',
    'Percentage': r'\d+\s*%\s*\d+',
    'Measurement': r'\d+\s*(m|cm|mm|km|ft|in|yd)',
    'Time': r'\d+\s*(s|m|h)',
    'Other': r'.+',
}

# Analyze incorrect answers and assign mistake types
df['Mistake Type'] = 'Other'  # default to 'Other' if no match
for i, row in df.iterrows():
    answer = str(row['Answer']).strip()
    for mistake_type, pattern in mistake_types.items():
        if re.match(pattern, answer):
            df.at[i, 'Mistake Type'] = mistake_type
            break

# Test the functions
attempts = 0
while attempts < 5:
    question, answer = get_next_question(df)
    print('Question:', question)
    print( answer)


    answer1 = input("Enter your answer: ")
    print('Answer:', answer)
    if answer1 == str(answer):
        print('Correct!')
    else:
        print('Incorrect!')
        update_mistakes(answer1, answer)

    attempts += 1
# Get user's most frequent mistake type
if 'Mistake Type' in df.columns:
    mistake_counts = df[df['Answer'] == False]['Mistake Type'].value_counts()
    most_frequent_type = mistake_counts.index[0] if not mistake_counts.empty else None

  # Get recommendations for the user's most frequent incorrect answers
if incorrect_answers:
    for incorrect_answer in incorrect_answers:
        matching_rows = df[df['Answer'] != incorrect_answer]
        if not matching_rows.empty:
            # Get a list of questions for the incorrect answer
            questions = matching_rows['Question'].tolist()
            # Create new ratings for the questions and add them to the dataset
            new_ratings = [(0, q, 0.001) for q in questions]
            new_data = Dataset.load_from_df(pd.DataFrame(new_ratings, columns=['Question', 'Answer', 'level']), reader)
            trainset = new_data.build_full_trainset()
            algo.fit(trainset)
            # Get recommendations for the questions
            for q in questions:
                item_id = trainset.to_inner_iid(q)
                neighbors = algo.get_neighbors(item_id, k=2)[1:]  # exclude the first item which is the question itself
                top_10_recommendations = [trainset.to_raw_iid(n) for n in neighbors]
                print(f"Recommendations for '{q}':")
                for r in top_10_recommendations:
                    print(f" - {r}")
        else:
            print(f"No questions found for incorrect answer: {incorrect_answer}")





Question: A bag contains 441 stigmatizations and 27 theologys. What is the total number of items in the bag?
468
Enter your answer: 468
Answer: 468
Correct!
Question: If you have 36 Pleurotuss and you did washed 13 Pleurotuss, how many Pleurotuss do you have left?
23
Enter your answer: 23
Answer: 23
Correct!
Question: 41 Idahos are in a box. If you did had 6 Idahos, how many Idahos are left in the box?
35
Enter your answer: 33
Answer: 35
Incorrect!
Question: A bag contains 81 Regencys and 58 Holothuroideas. What is the total number of items in the bag?
139
Enter your answer: 139
Answer: 139
Correct!
Question: John has 169 second trimester. He did painted 93 second trimesters. How many second trimesters does he have now?
76
Enter your answer: 77
Answer: 76
Incorrect!
Computing the cosine similarity matrix...
Done computing similarity matrix.
Recommendations for ' If Sally has 4 apples and she gives 1 to her friend, how many apples does Sally have left?':
 -  If you have 3 crayons and yo

In [ ]:
from surprise import SVD
from surprise import Dataset
from surprise import Reader

# Create a dictionary to keep track of incorrect answers
incorrect_answers = {}

def get_next_question(data):
    global incorrect_answers
    # If the user has answered any questions incorrectly before
    if incorrect_answers:
        # Find the most frequent incorrect answer
        most_frequent = max(set(incorrect_answers.keys()), key=incorrect_answers.get)
        # Filter the data by the most frequent incorrect answer
        filtered_data = data[data['Answer'] == most_frequent]
        # Check if the filtered data is empty
        if filtered_data.empty:
            # Choose a random question from the entire dataset
            row = data.sample()
            return row['Question'].iloc[0], row['Answer'].iloc[0]
        else:
            # Randomly choose a question from the filtered data
            row = filtered_data.sample()
            return row['Question'].iloc[0], row['Answer'].iloc[0]
    # If the user has not answered any questions incorrectly before, choose a random question
    else:
        row = data.sample()
        return row['Question'].iloc[0], row['Answer'].iloc[0]


# Define a function to update the incorrect answer dictionary
def update_incorrect_answers(answer, correct):
    global incorrect_answers
    if not correct:
        if answer in incorrect_answers:
            incorrect_answers[answer] += 1
        else:
            incorrect_answers[answer] = 1

# Load the data
reader = Reader(line_format='user item rating', sep=',')
data = Dataset.load_from_df(df[['Question', 'Answer', 'level']], reader)

# Define the algorithm
algo = SVD()

# Train the algorithm on the data
trainset = data.build_full_trainset()
algo.fit(trainset)

# Test the functions
attempts = 0
correctt = 0
while attempts < 5:
    question, answer = get_next_question(df)
    print('Question:', question)
    print('Answer:', answer)
    answer=str(answer)
    answer1 = input("Enter your answer: ")
    if answer1 == answer:
        correct = True
        print('Correct!')
        correctt += 1
    else:
        correct = False
        print('Incorrect!')
        update_incorrect_answers(answer, correct)

    attempts += 1

print('Most frequent incorrect answers:', incorrect_answers)

# Get recommendations for the user's most frequent incorrect answers
if incorrect_answers:
    most_frequent = max(set(incorrect_answers.keys()), key=incorrect_answers.get)
    questions = df[df['Answer'] != most_frequent]['Question'].tolist()
    new_ratings = [(0, q, 0.001) for q in questions]
    new_data = Dataset.load_from_df(pd.DataFrame(new_ratings, columns=['Question', 'Answer', 'level']), reader)
    trainset = new_data.build_full_trainset()
    algo.fit(trainset)

    for q in questions:
        item_id = trainset.to_inner_iid(q)
        predictions = algo.test([(0, item_id, 0.001)])
       
        top_10_recommendations = sorted(predictions, key=lambda x: x.est, reverse=True)[1:4]
        top_10_recommendations = [trainset.to_raw_iid(r.iid) for r in top_10_recommendations]
        print(f"Recommendations for '{q}':")
        for r in top_10_recommendations:
                    print(f" - {r}")


Question: There are 169 greetings in a bag. If you did painted 13 greetings, how many greetings are left in the bag?
Answer: 156
Enter your answer: 11
Incorrect!
Question: 169 Mirabiliss are in a box. If you did mowed 93 Mirabiliss, how many Mirabiliss are left in the box?
Answer: 76
Enter your answer: 44
Incorrect!
Question: There are 9 Chagalls in a bag. If you lost away 4 Chagalls, how many Chagalls are left in the bag?
Answer: 5
Enter your answer: 5
Correct!
Question: If you have 99 Freudian psychologys and you take out 1 Freudian psychologys, how many Freudian psychologys do you have left?
Answer: 98
Enter your answer: 98
Correct!
Question: A bag contains 324 cotton mills and 45 catchings. What is the total number of items in the bag?
Answer: 369
Enter your answer: 114
Incorrect!
Most frequent incorrect answers: {'156': 1, '76': 1, '369': 1}
Recommendations for ' If Sally has 4 apples and she gives 1 to her friend, how many apples does Sally have left?':
Recommendations for ' Ther